In [7]:
import tensorflow as tf
import numpy as np

In [8]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.3,0.4,0.9,0.4] #Bandit probabilites of winning or losing
num_bandits = len(bandits)

In [9]:
def pullBandit(bandit):
    #Get a random number.
    rand = np.random.random()
    if bandit > rand:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [10]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [13]:
total_episodes = 20 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
print("Total reward after 0 episodes: " + str(total_reward))

# Execute the TF session
sess = tf.Session();
init = tf.initialize_all_variables()
sess.run(init)
e = 0.1 #Set the chance of taking a random action.
i = 0
while i < total_episodes:        
    #Choose either a random action or one from our network.
    if np.random.rand(1) > e:
        action = sess.run(chosen_action)
    else:
        action = np.random.randint(num_bandits)    
        
    reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
    #Update the network.
    _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],
                                                                         action_holder:[action]})
    #Update our running tally of scores.
    total_reward[action] += reward
    
    i+=1
    if i % 1 == 0:
        print("Total reward after " + str(i) + " episodes: " + str(total_reward))    
    
print("\nThe agent guessed the best bandit is: " + str(np.argmax(ww)+1))
print("The known best bandit is: " + str(np.argmax(bandits)+1))

Total reward after 0 episodes: [0. 0. 0. 0.]
Total reward after 1 episodes: [-1.  0.  0.  0.]
Total reward after 2 episodes: [-1. -1.  0.  0.]
Total reward after 3 episodes: [-1. -1.  1.  0.]
Total reward after 4 episodes: [-1. -1.  2.  0.]
Total reward after 5 episodes: [-1. -1.  3.  0.]
Total reward after 6 episodes: [-1. -1.  4.  0.]
Total reward after 7 episodes: [-1. -1.  3.  0.]
Total reward after 8 episodes: [-1. -1.  4.  0.]
Total reward after 9 episodes: [-1. -1.  3.  0.]
Total reward after 10 episodes: [-1. -1.  2.  0.]
Total reward after 11 episodes: [-1. -1.  3.  0.]
Total reward after 12 episodes: [-1. -2.  3.  0.]
Total reward after 13 episodes: [-1. -2.  4.  0.]
Total reward after 14 episodes: [-1. -2.  5.  0.]
Total reward after 15 episodes: [-1. -2.  6.  0.]
Total reward after 16 episodes: [-1. -2.  7.  0.]
Total reward after 17 episodes: [-1. -2.  6.  0.]
Total reward after 18 episodes: [-1. -2.  5.  0.]
Total reward after 19 episodes: [-1. -2.  6.  0.]
Total reward a